In [1]:
import pandas as pd # python's data handling package
import numpy as np # python's scientific computing package
import matplotlib.pyplot as plt # python's plotting package
import openpyxl
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler

In [2]:
original_data = pd.read_excel('Original_Data.xlsx', engine='openpyxl', header=3) 
original_data = original_data.iloc[:,:-2]
print(original_data.head())

print(original_data['LotFrontage'].describe())
print(original_data.isnull().sum())

   Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD   

In [3]:
original_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2903,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,90500
2904,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,71000
2905,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,131000
2906,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000


In [4]:
features = ['LotFrontage', 'OverallQual', 'YearBuilt', 'LotArea', 'SalePrice', 'LotShape']
subset = original_data[features]

In [5]:
X = subset.drop('SalePrice', axis=1)  # Features
y = subset['SalePrice']  # Target variable
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# test_size=0.2 means 20% of the data is used for the test set, and 80% for the training set
# random_state is set to a fixed number for reproducibility


In [6]:
X_train

,LotFrontage,OverallQual,YearBuilt,LotArea,LotShape
1505,80.0,5,1966,9600,Reg
435,43.0,7,1996,10667,IR2
2785,90.0,6,1962,33120,IR3
237,NaN,7,1993,9453,IR1
2698,NaN,5,1967,8789,IR1
...,...,...,...,...,...
1638,NaN,8,2000,7500,Reg
1095,NaN,8,1986,3696,Reg
1130,90.0,6,1880,9900,Reg
1294,35.0,6,2005,3675,Reg


In [7]:
print(X_train.isnull().sum())

LotFrontage    391
OverallQual      0
YearBuilt        0
LotArea          0
LotShape         0
dtype: int64


In [8]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer

# Assuming X_train and X_val are pandas DataFrames
mses_mean_imputation = {}
mses_median_imputation = {}
mses_knn_imputation = {}
# Select numerical and categorical columns
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = ['LotShape']  # Specify categorical columns; add more if needed

# Define transformers
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
mean_imputer = SimpleImputer(strategy='mean')
median_imputer = SimpleImputer(strategy='median')
knn_imputer = KNNImputer(n_neighbors=5)

# Create column transformers for one-hot encoding of categorical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', one_hot_encoder, categorical_cols),
    ],
    remainder='passthrough'  # passthrough numerical columns as is
)

# Deep copy datasets
X_train_dc = X_train.copy(deep=True)
X_val_dc = X_val.copy(deep=True)

# Apply one-hot encoding to 'LotShape' in the deep copied data
X_train_encoded = preprocessor.fit_transform(X_train_dc)
X_val_encoded = preprocessor.transform(X_val_dc)

# Convert encoded arrays back to DataFrames
# Extract new column names for the one-hot encoded categories
columns_transformed = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols)
new_columns = list(columns_transformed) + [col for col in numerical_cols if col not in categorical_cols]

X_train_encoded_df = pd.DataFrame(X_train_encoded, columns=new_columns)
X_val_encoded_df = pd.DataFrame(X_val_encoded, columns=new_columns)

# Now apply imputation strategies on the one-hot encoded data
# Mean Imputation
X_train_mean = mean_imputer.fit_transform(X_train_encoded_df)
X_val_mean = mean_imputer.transform(X_val_encoded_df)

# Median Imputation
X_train_median = median_imputer.fit_transform(X_train_encoded_df)
X_val_median = median_imputer.transform(X_val_encoded_df)

# KNN Imputation
X_train_knn = knn_imputer.fit_transform(X_train_encoded_df)
X_val_knn = knn_imputer.transform(X_val_encoded_df)

scaler = StandardScaler()

# Standardize X_train_mean and overwrite
X_train_mean = scaler.fit_transform(X_train_mean)

# Standardize X_train_median and overwrite
# It's important to fit the scaler again for accurate mean and std
X_train_median = scaler.fit_transform(X_train_median)

# Standardize X_train_knn and overwrite
# Fit the scaler again for this dataset
X_train_knn = scaler.fit_transform(X_train_knn)

scaler_mean = StandardScaler().fit(X_train_mean)
scaler_median = StandardScaler().fit(X_train_median)
scaler_knn = StandardScaler().fit(X_train_knn)

# Apply the fitted scalers to the corresponding validation datasets
X_val_mean = scaler_mean.transform(X_val_mean)
X_val_median = scaler_median.transform(X_val_median)
X_val_knn = scaler_knn.transform(X_val_knn)


C:\Users\Kanish\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error as mse

# Assumed datasets
# X_train_mean, X_val_mean, y_train, y_val_mean (mean-imputed training and validation sets)
# X_train_median, X_val_median, y_train, y_val_median (median-imputed training and validation sets)
# X_train_knn, X_val_knn, y_train, y_val_knn (KNN-imputed training and validation sets)

# Define your alpha values
alphas = [0.01*1800, 0.02*1800, 0.03*1800, 0.04*1800, 0.05*1800, 0.075*1800, 0.1*1800, 0.2*1800, 0.4*1800]

# Placeholder for MSE results
mses = {
    'mean': [],
    'median': [],
    'knn': []
}

# Datasets dictionary for easier iteration
datasets = {
    'mean': (X_train_mean, X_val_mean, y_val),
    'median': (X_train_median, X_val_median, y_val),
    'knn': (X_train_knn, X_val_knn, y_val)
}

# Loop over each alpha
for alpha in alphas:
    print(f"Alpha: {alpha}")
    
    # Evaluate each dataset
    for imputation_type, (X_train, X_val, y_val) in datasets.items():
        ridge = Ridge(alpha=alpha)
        ridge.fit(X_train, y_train)  # Fit model using training data
        pred = ridge.predict(X_val)  # Predict on respective validation set
        mse_value = mse(y_val, pred)  # Calculate MSE for the validation set
        mses[imputation_type].append(mse_value)  # Store MSE
        print(f"{imputation_type} MSE: {mse_value}")


Alpha: 18.0
mean MSE: 3.972317282051605e+16
median MSE: 3.976775808178215e+16
knn MSE: 3.6935641254552456e+16
Alpha: 36.0
mean MSE: 3.943982656612004e+16
median MSE: 3.948336608328084e+16
knn MSE: 3.669678458580654e+16
Alpha: 54.0
mean MSE: 3.915514114220067e+16
median MSE: 3.919769543331963e+16
knn MSE: 3.645576499108497e+16
Alpha: 72.0
mean MSE: 3.886956793517639e+16
median MSE: 3.891119320790505e+16
knn MSE: 3.62130340102007e+16
Alpha: 90.0
mean MSE: 3.858351001359455e+16
median MSE: 3.86242585032014e+16
knn MSE: 3.5968997489695116e+16
Alpha: 135.0
mean MSE: 3.786851119746047e+16
median MSE: 3.790727290869035e+16
knn MSE: 3.5355498438057844e+16
Alpha: 180.0
mean MSE: 3.715726496484776e+16
median MSE: 3.719429426905006e+16
knn MSE: 3.4740781788299264e+16
Alpha: 360.0
mean MSE: 3.4406934989960996e+16
median MSE: 3.443885792022477e+16
knn MSE: 3.233065477834054e+16
Alpha: 720.0
mean MSE: 2.9580372870224932e+16
median MSE: 2.960666285471789e+16
knn MSE: 2.8008368738219396e+16


Modify ^ so that it only selects the best mean media knn alpha for each run - ask chatgpt to do it

In [10]:

lr1 = LinearRegression()
lr2 = LinearRegression()
lr3 = LinearRegression()
lr1.fit(X_train_mean, y_train)
pred_lr = lr1.predict(X_val_mean)
mses_mean_imputation['Linear Regression'] = mse(y_val, pred_lr)

lr2.fit(X_train_median, y_train)
pred_lr = lr2.predict(X_val_median)
mses_median_imputation['Linear Regression'] = mse(y_val, pred_lr)

lr3.fit(X_train_knn, y_train)
pred_lr = lr3.predict(X_val_knn)
mses_knn_imputation['Linear Regression'] = mse(y_val, pred_lr)


In [11]:
ridge = Ridge(alpha=0.1 * len(X_train))  # Adjust alpha as necessary
ridge.fit(X_train_mean, y_train)
pred_ridge = ridge.predict(X_val_mean)
mses_mean_imputation['Ridge'] = mse(y_val, pred_ridge)

ridge.fit(X_train_median, y_train)
pred_ridge = ridge.predict(X_val_median)
mses_median_imputation['Ridge'] = mse(y_val, pred_ridge)

ridge.fit(X_train_knn, y_train)
pred_ridge = ridge.predict(X_val_knn)
mses_knn_imputation['Ridge'] = mse(y_val, pred_ridge)

In [12]:
# from sklearn.linear_model import Lasso
# lasso = Lasso(alpha=0.05)
# lasso.fit(X_train, y_train)

In [13]:
# coeffs = pd.DataFrame(
#     [
#         ['intercept'] + list(X_train.columns),
#         list(lasso.intercept_) + list(lasso.coef_)
#     ]
# ).transpose().set_index(0)
# coeffs

In [14]:
lasso = Lasso(alpha=0.05)  # Adjust alpha as necessary
lasso.fit(X_train_mean, y_train)
pred_lasso = lasso.predict(X_val_mean)
mses_mean_imputation['Lasso wiht Lot Shape'] = mse(y_val, pred_lasso)

lasso.fit(X_train_median, y_train)
pred_lasso = lasso.predict(X_val_median)
mses_median_imputation['Lasso with Lot Shape'] = mse(y_val, pred_lasso)

lasso.fit(X_train_knn, y_train)
pred_lasso = lasso.predict(X_val_knn)
mses_knn_imputation['Lasso with Lot Shape'] = mse(y_val, pred_lasso)

In [15]:
# lasso.fit(ls_X_train_mean, y_train)
# ls_pred_lasso_mean = lasso.predict(ls_X_val_mean)
# mses_mean_imputation['Lasso with Lot Shape'] = mse(y_val, ls_pred_lasso_mean)

# lasso.fit(ls_X_train_median, y_train)
# ls_pred_lasso_median = lasso.predict(ls_X_val_median)
# mses_median_imputation['Lasso with Lot Shape'] = mse(y_val, ls_pred_lasso_median)

# lasso.fit(ls_X_train_knn, y_train)
# ls_pred_lasso_knn = lasso.predict(ls_X_val_knn)
# mses_knn_imputation['Lasso with Lot Shape'] = mse(y_val, ls_pred_lasso_knn)

In [16]:
print("MSE with Mean Imputation:")
for model, mse_val in mses_mean_imputation.items():
    print(f"{model}: {mse_val}")
    
print("\nMSE with Median Imputation:")
for model, mse_val in mses_median_imputation.items():
    print(f"{model}: {mse_val}")
    
print("\nMSE with KNN Imputation:")
for model, mse_val in mses_knn_imputation.items():
    print(f"{model}: {mse_val}")

MSE with Mean Imputation:
Linear Regression: 1.0025264998040222e+29
Ridge: 3.63352468450325e+16
Lasso wiht Lot Shape: 4.000492921298768e+16

MSE with Median Imputation:
Linear Regression: 2.778838941634649e+28
Ridge: 3.63705190740979e+16
Lasso with Lot Shape: 4.005063487902393e+16

MSE with KNN Imputation:
Linear Regression: 1.915132397064912e+32
Ridge: 3.402552714348955e+16
Lasso with Lot Shape: 3.717197750987913e+16


Make a copy of this entire process and redo it when including Lot Shape 